<a href="https://colab.research.google.com/github/AnitaTasnim/WrongWayDrivingCode/blob/main/WrongWayDriving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Start

In [1]:
!git clone https://github.com/Usmankhujaev/Wrong-direction-drivers-detection.git
%cd Wrong-direction-drivers-detection


Cloning into 'Wrong-direction-drivers-detection'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 88 (delta 12), reused 4 (delta 4), pack-reused 73 (from 1)
Receiving objects: 100% (88/88), 1.53 MiB | 41.14 MiB/s, done.
Resolving deltas: 100% (33/33), done.
/content/Wrong-direction-drivers-detection


In [2]:
!ls


detector_car_person.py	model_data	    requirenment.yml  train.py
KalmanFilter.py		object_tracking.py  result	      yolo3
kmeans.py		README.md	    tracker.py


In [3]:
!wget https://pjreddie.com/media/files/yolov3.weights -O model_data/yolov3.weights


--2025-12-17 21:15:37--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 172.67.185.199, 104.21.88.156, 2606:4700:3030::ac43:b9c7, ...
Connecting to pjreddie.com (pjreddie.com)|172.67.185.199|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://data.pjreddie.com/files/yolov3.weights [following]
--2025-12-17 21:15:37--  https://data.pjreddie.com/files/yolov3.weights
Resolving data.pjreddie.com (data.pjreddie.com)... 172.67.185.199, 104.21.88.156, 2606:4700:3030::ac43:b9c7, ...
Connecting to data.pjreddie.com (data.pjreddie.com)|172.67.185.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘model_data/yolov3.weights’

model_data/yolov3.w 100%[===================>] 236.52M  36.3MB/s    in 5.9s    

2025-12-17 21:15:43 (39.8 MB/s) - ‘model_data/yolov3.weights’ saved [248007048/248007048]



In [4]:
!ls model_data


car_class_2.txt  yolo_anchors.txt  yolov3.weights


In [5]:
!ls result


algorithm2.png
result.png
wrong_direction_20191118_181432_2_01.jpg
wrong_direction_20191118_181432_3_01.jpg
wrong_direction_20191118_181433_3_00.jpg
wrong_direction_20191118_181433_3_01.jpg
wrong_direction_20191118_181501_2_01.jpg
wrong_direction_20191118_181502_2_01.jpg
wrong_direction_20191118_181502_3_00.jpg
wrong_direction_20191118_181503_3_00.jpg


In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
# ===== CLEAN ENV FOR COLAB =====
'''!pip uninstall -y numpy scipy
!pip install --no-cache-dir numpy==1.26.4 scipy==1.11.4
!pip install opencv-python filterpy matplotlib pillow ultralytics
'''

'!pip uninstall -y numpy scipy\n!pip install --no-cache-dir numpy==1.26.4 scipy==1.11.4\n!pip install opencv-python filterpy matplotlib pillow ultralytics\n'

In [8]:
import numpy as np
print("NumPy:", np.__version__)
print(np.char)

from scipy.optimize import linear_sum_assignment
print("SciPy OK")


NumPy: 2.0.2
<module 'numpy.char' from '/usr/local/lib/python3.12/dist-packages/numpy/char/__init__.py'>
SciPy OK


In [9]:
'''!pip install -q --force-reinstall \
numpy==1.26.4 \
scipy==1.11.4 \
opencv-python \
opencv-contrib-python \
pillow \
matplotlib \
filterpy \
tensorflow==2.16.1 \
keras
'''

'!pip install -q --force-reinstall numpy==1.26.4 scipy==1.11.4 opencv-python opencv-contrib-python pillow matplotlib filterpy tensorflow==2.16.1 keras\n'

#kalman_filter.py

In [10]:
import numpy as np
from numpy import dot

class KalmanFilterClass(object):
    def __init__(self):
        self.dt = 5e-3
        self.A = np.array([[1,0],[0,1]])
        self.u = np.zeros((2,1))
        self.b = np.array([[0],[255]])
        self.P = np.diag((3.0, 3.0))
        self.F = np.array([[1.0, self.dt], [0.0, 1.0]])
        self.Q = np.eye(self.u.shape[0])
        self.R = np.eye(self.b.shape[0])
        self.lastResult = np.array([[0], [255]])

    def predict(self):
        self.u = np.round(dot(self.F, self.u))
        self.P = dot(self.F, dot(self.P, self.F.T)) + self.Q
        self.lastResult = self.u
        return self.u

    def correct(self, b, flag):
        if not flag:
            self.b = self.lastResult
        else:
            self.b = b
        C = dot(self.A, dot(self.P, self.A.T))+self.R
        K = dot(self.P, dot(self.A.T, np.linalg.inv(C)))
        self.u = np.round(self.u + dot(K, (self.b - dot(self.A, self.u))))
        self.P = self.P - dot(K, dot(C, K.T))
        self.lastResult = self.u
        return self.u

#tracker

In [11]:


# Import python libraries
import numpy as np
#from KalmanFilter import KalmanFilterClass

from scipy.optimize import linear_sum_assignment


class Track(object):


    def __init__(self, prediction, trackIdCount):

        self.track_id = trackIdCount  # identification of each track object
        self.KF = KalmanFilterClass()  # KF instance to track this object
        self.prediction = np.asarray(prediction)  # predicted centroids (x,y)
        self.skipped_frames = 0  # number of frames skipped undetected
        self.trace = []  # trace path


class Tracker(object):


    def __init__(self, dist_thresh, max_frames_to_skip, max_trace_length,
                 trackIdCount):

        self.dist_thresh = dist_thresh
        self.max_frames_to_skip = max_frames_to_skip
        self.max_trace_length = max_trace_length
        self.tracks = []
        self.trackIdCount = trackIdCount

    def Update(self, detections):

        # Create tracks if no tracks vector found
        if (len(self.tracks) == 0):
            for i in range(len(detections)):
                track = Track(detections[i], self.trackIdCount)
                self.trackIdCount += 1
                self.tracks.append(track)

        # Calculate cost using sum of square distance between
        # predicted vs detected centroids
        N = len(self.tracks)
        M = len(detections)
        cost = np.zeros(shape=(N, M))   # Cost matrix
        for i in range(len(self.tracks)):
            for j in range(len(detections)):
                try:
                    diff = self.tracks[i].prediction - detections[j]
                    distance = np.sqrt(diff[0][0]*diff[0][0] +
                                       diff[1][0]*diff[1][0])
                    cost[i][j] = distance
                except:
                    pass

        # Let's average the squared ERROR
        cost = (0.5) * cost
        # Using Hungarian Algorithm assign the correct detected measurements
        # to predicted tracks
        assignment = []
        for _ in range(N):
            assignment.append(-1)
        row_ind, col_ind = linear_sum_assignment(cost)
        for i in range(len(row_ind)):
            assignment[row_ind[i]] = col_ind[i]

        # Identify tracks with no assignment, if any
        un_assigned_tracks = []
        for i in range(len(assignment)):
            if (assignment[i] != -1):
                # check for cost distance threshold.
                # If cost is very high then un_assign (delete) the track
                if (cost[i][assignment[i]] > self.dist_thresh):
                    assignment[i] = -1
                    un_assigned_tracks.append(i)
                pass
            else:
                self.tracks[i].skipped_frames += 1

        # If tracks are not detected for long time, remove them
        del_tracks = []
        for i in range(len(self.tracks)):
            if (self.tracks[i].skipped_frames > self.max_frames_to_skip):
                del_tracks.append(i)
        if len(del_tracks) > 0:  # only when skipped frame exceeds max
            for id in del_tracks:
                if id < len(self.tracks):
                    del self.tracks[id]
                    del assignment[id]
                else:
                    print("ERROR: id is greater than length of tracks")

        # Now look for un_assigned detects
        un_assigned_detects = []
        for i in range(len(detections)):
                if i not in assignment:
                    un_assigned_detects.append(i)

        # Start new tracks
        if(len(un_assigned_detects) != 0):
            for i in range(len(un_assigned_detects)):
                track = Track(detections[un_assigned_detects[i]],
                              self.trackIdCount)
                self.trackIdCount += 1
                self.tracks.append(track)

        # Update KalmanFilter state, lastResults and tracks trace
        for i in range(len(assignment)):
            self.tracks[i].KF.predict()

            if(assignment[i] != -1):
                self.tracks[i].skipped_frames = 0
                self.tracks[i].prediction = self.tracks[i].KF.correct(
                                            detections[assignment[i]], 1)
            else:
                self.tracks[i].prediction = self.tracks[i].KF.correct(
                                            np.array([[0], [0]]), 0)

            if(len(self.tracks[i].trace) > self.max_trace_length):
                for j in range(len(self.tracks[i].trace) -
                               self.max_trace_length):
                    del self.tracks[i].trace[j]

            self.tracks[i].trace.append(self.tracks[i].prediction)
            self.tracks[i].KF.lastResult = self.tracks[i].prediction



#YOLO

##centroidtracker.py

In [12]:
from scipy.spatial import distance as dist
from collections import OrderedDict
import numpy as np
class CentroidTracker():
    def __init__(self, maxDisappeared=50):
        self.nextObjectID = 0
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()
        self.maxDisappeared = maxDisappeared
    def register(self, centroid):
        self.objects[self.nextObjectID]=centroid
        self.disappeared[self.nextObjectID]=0
        self.nextObjectID +=1
    def deregister(self, objectID):
        del self.objects[objectID]
        del self.disappeared[objectID]
    def update(self, rects):
        if len(rects)==0:
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID]+=1
                if self.disappeared[objectID]>self.maxDisappeared:
                    self.deregister(objectID)
            return self.objects
        inputCentroids = np.zeros((len(rects), 2), dtype="int")
        for (i, (xmin, ymin, xmax, ymax)) in enumerate(rects):
            mid_x = int((xmin+xmax) / 2)
            mid_y = int((ymin+ymax) / 2)
            inputCentroids[i] = (mid_x, mid_y)
        if len(self.objects)==0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i])
        else:
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())
            D = dist.cdist(np.array(objectCentroids), inputCentroids)
            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]
            usedRows = set()
            usedCols = set()
            for (row, col) in zip(rows, cols):
                if row in usedRows or col in usedCols:
                    continue
                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.disappeared[objectID] = 0
                usedRows.add(row)
                usedCols.add(col)
                unusedRows = set(range(0, D.shape[0])).difference(usedRows)
                unusedCols = set(range(0, D.shape[1])).difference(usedCols)
                if D.shape[0] >= D.shape[1]:
                    for row in unusedRows:
                        objectID=objectIDs[row]
                        self.disappeared[objectID] += 1
                        if self.disappeared[objectID]> self.maxDisappeared:
                            self.deregister(objectID)
                        else:
                            for col in unusedCols:
                                self.register(inputCentroids[col])
        return self.objects

##utils.py

In [13]:
"""Miscellaneous utility functions."""

from functools import reduce

from PIL import Image
import cv2
import numpy as np
from matplotlib.colors import rgb_to_hsv, hsv_to_rgb

def compose(*funcs):
    """Compose arbitrarily many functions, evaluated left to right.

    Reference: https://mathieularose.com/function-composition-in-python/
    """
    # return lambda x: reduce(lambda v, f: f(v), funcs, x)
    if funcs:
        return reduce(lambda f, g: lambda *a, **kw: g(f(*a, **kw)), funcs)
    else:
        raise ValueError('Composition of empty sequence not supported.')

def letterbox_image(image, size):
    '''resize image with unchanged aspect ratio using padding'''
    iw, ih = image.size
    w, h = size
    scale = min(w/iw, h/ih)
    nw = int(iw*scale)
    nh = int(ih*scale)

    image = image.resize((nw,nh), Image.BICUBIC)
    new_image = Image.new('RGB', size, (128,128,128))
    new_image.paste(image, ((w-nw)//2, (h-nh)//2))
    return new_image

def rand(a=0, b=1):
    return np.random.rand()*(b-a) + a

def get_random_data(annotation_line, input_shape, random=True, max_boxes=20, jitter=.3, hue=.1, sat=1.5, val=1.5, proc_img=True):
    '''random preprocessing for real-time data augmentation'''
    line = annotation_line.split()
    image = Image.open(line[0])
    iw, ih = image.size
    h, w = input_shape
    box = np.array([np.array(list(map(int,box.split(',')))) for box in line[1:]])

    if not random:
        # resize image
        scale = min(w/iw, h/ih)
        nw = int(iw*scale)
        nh = int(ih*scale)
        dx = (w-nw)//2
        dy = (h-nh)//2
        image_data=0
        if proc_img:
            image = image.resize((nw,nh), Image.BICUBIC)
            new_image = Image.new('RGB', (w,h), (128,128,128))
            new_image.paste(image, (dx, dy))
            image_data = np.array(new_image)/255.

        # correct boxes
        box_data = np.zeros((max_boxes,5))
        if len(box)>0:
            np.random.shuffle(box)
            if len(box)>max_boxes: box = box[:max_boxes]
            box[:, [0,2]] = box[:, [0,2]]*scale + dx
            box[:, [1,3]] = box[:, [1,3]]*scale + dy
            box_data[:len(box)] = box

        return image_data, box_data

    # resize image
    new_ar = w/h * rand(1-jitter,1+jitter)/rand(1-jitter,1+jitter)
    scale = rand(.25, 2)
    if new_ar < 1:
        nh = int(scale*h)
        nw = int(nh*new_ar)
    else:
        nw = int(scale*w)
        nh = int(nw/new_ar)
    image = image.resize((nw,nh), Image.BICUBIC)

    # place image
    dx = int(rand(0, w-nw))
    dy = int(rand(0, h-nh))
    new_image = Image.new('RGB', (w,h), (128,128,128))
    new_image.paste(image, (dx, dy))
    image = new_image

    # flip image or not
    flip = rand()<.5
    if flip: image = image.transpose(Image.FLIP_LEFT_RIGHT)

    # distort image
    hue = rand(-hue, hue)
    sat = rand(1, sat) if rand()<.5 else 1/rand(1, sat)
    val = rand(1, val) if rand()<.5 else 1/rand(1, val)
    x = rgb_to_hsv(np.array(image)/255.)
    x[..., 0] += hue
    x[..., 0][x[..., 0]>1] -= 1
    x[..., 0][x[..., 0]<0] += 1
    x[..., 1] *= sat
    x[..., 2] *= val
    x[x>1] = 1
    x[x<0] = 0
    image_data = hsv_to_rgb(x) # numpy array, 0 to 1

    # correct boxes
    box_data = np.zeros((max_boxes,5))
    if len(box)>0:
        np.random.shuffle(box)
        box[:, [0,2]] = box[:, [0,2]]*nw/iw + dx
        box[:, [1,3]] = box[:, [1,3]]*nh/ih + dy
        if flip: box[:, [0,2]] = w - box[:, [2,0]]
        box[:, 0:2][box[:, 0:2]<0] = 0
        box[:, 2][box[:, 2]>w] = w
        box[:, 3][box[:, 3]>h] = h
        box_w = box[:, 2] - box[:, 0]
        box_h = box[:, 3] - box[:, 1]
        box = box[np.logical_and(box_w>1, box_h>1)] # discard invalid box
        if len(box)>max_boxes: box = box[:max_boxes]
        box_data[:len(box)] = box

    return image_data, box_data

##model compatibility

In [14]:
#!pip uninstall -y tensorflow keras jax jaxlib ml-dtypes
#!pip install tensorflow==2.10.1 keras==2.10.0 ml-dtypes==0.5.0


In [15]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"


In [16]:
'''!pip uninstall -y pillow
!pip install pillow==10.2.0'''

'!pip uninstall -y pillow\n!pip install pillow==10.2.0'

In [17]:
'''from ultralytics import YOLO'''

'from ultralytics import YOLO'

#detector_car_person.py

In [19]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.5 MB/s eta 0:00:00


In [22]:
# detector_car_person_yolov8.py
import cv2
import datetime as dt
import numpy as np
from ultralytics import YOLO
#from tracker import Tracker  # Your existing Tracker class

def resize(img, scale=200):
    width = int(img.shape[1] * scale / 100)
    height = int(img.shape[0] * scale / 100)
    return cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)




def detect_video(video_path, model_path="yolov8n.pt", output_path=""):
    model = YOLO(model_path)
    tracker = Tracker(30, 0, 6, 0)  # Your tracker

    vid = cv2.VideoCapture(video_path)
    width, height = 400, 300
    pause = False

    if output_path:
        fourcc = int(vid.get(cv2.CAP_PROP_FOURCC))
        fps = vid.get(cv2.CAP_PROP_FPS)
        out = cv2.VideoWriter(output_path, fourcc, fps,
                              (int(0.78*width)-int(0.109*width), height-int(height*0.0608)))

    track_colors = [(255,0,0), (0,255,0), (0,0,255), (255,255,0),
                    (0,255,255), (255,0,255)]

    while True:
        ret, frame = vid.read()
        if not ret:
            break

        orig_frame = frame.copy()
        frame = cv2.resize(frame, (width, height))
        frame = frame[int(0.0608*height):height, int(0.109*width):int(0.78*width)]
        h, w, _ = frame.shape

        results = model.predict(frame, imgsz=416, conf=0.3, iou=0.4)[0]

        boxes = results.boxes.xyxy.cpu().numpy()  # x1, y1, x2, y2
        class_ids = results.boxes.cls.cpu().numpy().astype(int)
        scores = results.boxes.conf.cpu().numpy()

        centroids = []
        all_classes = []

        for i, box in enumerate(boxes):
            x1, y1, x2, y2 = box.astype(int)
            mid_x, mid_y = int((x1+x2)/2), int((y1+y2)/2)
            centroids.append(np.array([[mid_y],[mid_x]]))
            predicted_class = model.names[class_ids[i]]
            all_classes.append(predicted_class)

            if predicted_class.lower() == "person":
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0,0,255), 1)
                cv2.putText(frame, predicted_class, (x1, y1),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 1)

        # Update tracker
        if centroids:
            tracker.Update(centroids)
            for t in tracker.tracks:
                if len(t.trace) > 1:
                    for j in range(len(t.trace)-1):
                        y1, x1 = t.trace[j][0][0], t.trace[j][1][0]
                        y2, x2 = t.trace[j+1][0][0], t.trace[j+1][1][0]
                        clr = t.track_id
                        cv2.arrowedLine(frame, (int(x1), int(y1)), (int(x2), int(y2)),
                                        track_colors[clr % len(track_colors)],
                                        line_type=cv2.LINE_AA, thickness=1)

        cv2.putText(frame, f"Count: {len(centroids)}", (3,35),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,255,0), 1)

        #cv2.imshow("result", frame)
        cv2_imshow(frame)
        if output_path:
            out.write(frame)

        import time
        time.sleep(0.03)  # ~30 FPS
        '''key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        if key == 112:  # 'p' pause
            pause = not pause
            while pause:
                key2 = cv2.waitKey(30) & 0xFF
                if key2 == 112:
                    pause = False
                    break'''

    vid.release()
    if output_path:
        out.release()
    cv2.destroyAllWindows()

    '''

# Example usage
if __name__ == "__main__":
    detect_video("test_video.mp4", model_path="yolov8n.pt", output_path="output.mp4")'''


#object_tracking.py

In [23]:
from google.colab.patches import cv2_imshow


In [24]:
import cv2
from ultralytics import YOLO
import os

def detect_video_inline(
    video_path,
    model_path="yolov8n.pt",
    output_video_path="output.mp4",
    output_frames_dir="/content/output_frames",
    save_every_n=1   # save every frame (increase to reduce storage)
):
    os.makedirs(output_frames_dir, exist_ok=True)

    model = YOLO(model_path)
    cap = cv2.VideoCapture(video_path)

    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    frame_id = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model.predict(frame, conf=0.3, iou=0.4, verbose=False)[0]
        annotated = results.plot()

        # Save video
        out.write(annotated)

        # Save frames
        if frame_id % save_every_n == 0:
            frame_path = os.path.join(
                output_frames_dir, f"frame_{frame_id:06d}.jpg"
            )
            cv2.imwrite(frame_path, annotated)

        frame_id += 1
        if frame_id % 100 == 0:
            print(f"Processed {frame_id} frames")

    cap.release()
    out.release()
    print("✅ Video saved to:", output_video_path)
    print("✅ Frames saved to:", output_frames_dir)


In [25]:
detect_video_inline(
    video_path="/content/drive/MyDrive/Jakir_Sir_office/WWD/test_videos/car.mp4",
    model_path="yolov8n.pt",
    output_video_path="/content/drive/MyDrive/Jakir_Sir_office/WWD/output/output_car.mp4",
    output_frames_dir="/content/drive/MyDrive/Jakir_Sir_office/WWD/output/frames",
    save_every_n=5   # save 1 frame every 5 frames
)


Processed 100 frames
Processed 200 frames
Processed 300 frames
Processed 400 frames
Processed 500 frames
Processed 600 frames
Processed 700 frames
Processed 800 frames
Processed 900 frames
Processed 1000 frames
Processed 1100 frames
Processed 1200 frames
Processed 1300 frames
Processed 1400 frames
Processed 1500 frames
Processed 1600 frames
Processed 1700 frames
Processed 1800 frames
Processed 1900 frames
Processed 2000 frames
Processed 2100 frames
Processed 2200 frames
Processed 2300 frames
Processed 2400 frames
Processed 2500 frames
Processed 2600 frames
Processed 2700 frames
Processed 2800 frames
Processed 2900 frames
Processed 3000 frames
Processed 3100 frames
Processed 3200 frames
Processed 3300 frames
Processed 3400 frames
Processed 3500 frames
Processed 3600 frames
Processed 3700 frames
Processed 3800 frames
Processed 3900 frames
Processed 4000 frames
Processed 4100 frames
Processed 4200 frames
Processed 4300 frames
Processed 4400 frames
Processed 4500 frames
Processed 4600 fram

#then